## 最小ID

In [1]:
import numpy as np
from commons import how_long
from memory_profiler import profile

In [2]:
## 保证每次生成的一样
np.random.seed(1)
# 生成20万大小的id list
n=200000
ids = np.random.choice(range(n+2),n,replace=False)
if 0 not in ids:
    ids.append(0)
print(ids[1:4])

[171553   5506  38370]


>[171553   5506  38370]

In [3]:
## 暴力方法
@how_long
def min_id(ids):
    '''
    暴力方法
    ids:list of ids
    
    return min_id for create
    '''
    for i in range(len(ids)):
        if not_in(i,ids):
            return i
#     print(len(ids))
    return len(ids)

@how_long
def min_id_with_not_in(ids):
    for i in range(len(ids)):
        if i not in ids:
            return i
#     print(len(ids))
    return len(ids)
    

def not_in(i,ids):
    '''
    检查i是否在ids中，原本可以用
    if i not in ids
    为了保证使用最原始方法测试，所以自己用for重新写一个
    '''
    for one_id in ids:
        if i == one_id:
            return False
    return True

## 改进一

@how_long
def min_id_change_1(ids):
    '''
    改进一
    '''
    id_index = [False] * len(ids)
    for i in ids:
        if i < len(ids):
            id_index[i]=True
    for ii in range(len(id_index)):
        if id_index[ii] == False:
            return ii
#     print(len(ids))
    return len(ids)



## 改进二


def min_id_change_2_sub(l,u,ids):
    '''
    改进二
    '''
    if len(ids)==0:
        return l
    else:     
        middle = np.floor((l+u)/2)
        small_set=[]
        other_set=[]
        length = len(ids)
        for i in range(length):
            the_number,ids = ids[-1], ids[:-1]
            if the_number <= middle:
                small_set.append(the_number)
            else:
                other_set.append(the_number)
             
    if len(small_set) == middle+1-l:
        return min_id_change_2_sub(middle+1,middle+len(other_set),other_set)
    else:
        return min_id_change_2_sub(l,middle,small_set)
    return result
        


@how_long
def min_id_change_2(ids):
    '''
    递归方法
    '''
    result = min_id_change_2_sub(0,len(ids)-1,ids)
#     print(result)
    return result


## 改进二，不是用递归，而转化成迭代
@how_long
def min_id_change_2_iteration(ids):
    '''
    迭代方法
    '''
    n=len(ids)
    l=0
    u=n-1
    
    while n:
        left=0
        middle = np.floor((u+l)/2)
#         print(n,left,middle,l,u)
        for right in range(int(l),int(l+n)):
            if ids[right] <= middle:
                tmp = ids[right]
                ids[right]=ids[left+l]
                ids[left+l] =tmp
                left += 1
#             print(ids)
#         print('------')
#         print(ids)
#         print(left+l,middle - l + 1)
        if left == middle - l + 1:
#             print('left')
            l = int(middle + 1)
            u = u
            n = n-left
        else:
#             print('right')
            u = middle
            n = left
    return l



In [4]:
## 自己用for循环模仿 not in 
%memit print(min_id(ids))

min_id() cost:101690.6300 ms!
5192
peak memory: 69.01 MiB, increment: 0.05 MiB


In [5]:
## 使用 if i not in ids
%memit print(min_id_with_not_in(ids))

min_id_with_not_in() cost:410.6679 ms!
5192
peak memory: 69.04 MiB, increment: 0.02 MiB


In [6]:
## 改进一
%memit print(min_id_change_1(ids))

min_id_change_1() cost:66.8340 ms!
5192
min_id_change_1() cost:70.8082 ms!
5192
peak memory: 65.98 MiB, increment: 0.00 MiB


In [7]:
## 改进二 递归方法
%memit print(min_id_change_2(ids))

min_id_change_2() cost:36050.0872 ms!
5192.0
peak memory: 76.84 MiB, increment: 10.86 MiB


In [8]:
## 改进二 迭代方法
%memit print(min_id_change_2_iteration(ids))

min_id_change_2_iteration() cost:727.1988 ms!
5192
peak memory: 68.50 MiB, increment: 0.00 MiB


书中提到了，改进二，由于使用了递归，所以有一个调用的栈的大小消耗在内存中，所以会使得内存的消耗增大。

这还只是一个20万的id集合，而且查询的id只是5000左右，结果已经有很大差异了。

如果优化的好，迭代方法的确要比递归方法好。

<font color='red'>而且有一点，就是python中的内置方法`in`应该不是简单的for循环，或者是用c编译的，速度要快很多。所以在使用过程中，能使用已有的内置方法，最好就直接使用，应该会比自己写的要快。</font>

## 丑数